In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from datasets import Dataset, load_from_disk
import pytorch_lightning as pl
import sys
import os
# Add the directory containing lit_sam_model.py to the Python path
sys.path.append(os.path.abspath("../"))
from model.adapterModel import LitSamModel
from utils.statistics import calculate_correlation
from torch.utils.data import DataLoader
from model.samDataset import SAMDataset3
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   # maps logical GPU 0 -> physical GPU 1
from helperFunctions import *

In [ ]:
import yaml
import os
from pathlib import Path

# 1. Get the path of the script
current_file = Path(__file__).resolve() # src/training/your_script.py

# 2. Go up one level to 'src', then into 'config'
config_path = current_file.parent.parent / "config" / "config_general.yaml"

# 3. Load the YAML
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

# 4. Resolve the root of the project (one level above 'src')
# This ensures that "./data" in the YAML is interpreted relative to the Project_Root
PROJECT_ROOT = current_file.parent.parent.parent
os.chdir(PROJECT_ROOT) 

# Extract paths from YAML
DATA_DIR = config['paths']['data']
CHECKPOINT_DIR = config['paths']['checkpoints']
SAM_CHECKPOINT = config['paths']['sam_checkpoint']

In [ ]:
#Load the test dataset

test_dataset = load_from_disk(os.path.join(DATA_DIR, 'datasetTestFinal'))

In [ ]:
from transformers import SamModel, SamConfig, SamProcessor
import torch
from model.inputTypes import InputTypes


sam_checkpoint = os.path.join(CHECKPOINT_DIR, 'sam-float-adapter-newbase-epoch=57-val_loss=0.224-val_iou=0.592.ckpt')
# Create an instance of the model architecture with the loaded configuration
#model = LitSamModel.load_from_checkpoint(sam_checkpoint, model_name="vit-b", normalize = True, adapt_patch_embed = False, input_type = InputTypes.Normal)
model = LitSamModel(model_name="vit_b", normalize=True, learning_rate=1e-5, input_type=InputTypes.Normal, adapt = False)
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

# set the device to cuda if available, otherwise use cpu
device = "cuda" if torch.cuda.is_available() else "cpu"
#model.to(device)

In [ ]:
def calculate_iou(mask1, mask2):

    # Ensure the masks are PyTorch tensors
    if isinstance(mask1, np.ndarray):
        mask1 = torch.tensor(mask1)
    if isinstance(mask2, np.ndarray):
        mask2 = torch.tensor(mask2)
        
    # Ensure the masks are binary
    mask1 = mask1 > 0
    mask2 = mask2 > 0
    
    # Calculate the intersection and union
    intersection = torch.logical_and(mask1, mask2)
    union = torch.logical_or(mask1, mask2)
    
    # Compute the IoU
    iou = torch.sum(intersection).float() / torch.sum(union).float()
    
    return iou, intersection, union

In [ ]:
# Create an instance of the SAMDataset
test_dataset_sam = SAMDataset3(dataset=test_dataset, processor=processor, augment=False, test = True, type = InputTypes.Normal)

# Create a DataLoader instance for the validation dataset
test_dataloader = DataLoader(test_dataset_sam, batch_size=5, shuffle=False, num_workers=8)

In [ ]:
# Create a trainer
trainer = pl.Trainer(accelerator='gpu', devices=1)

# Run the evaluation
trainer.test(model, dataloaders=test_dataloader)

In [ ]:
# Access the stored results
test_results = model.test_results
ground_truth_masks = test_results['ground_truth_masks']
predicted_masks = test_results['predicted_masks']
individual_ious = test_results['individual_ious']
bboxes = test_results['bboxes']

In [ ]:
predicted_masks = np.concatenate(predicted_masks, axis=0)
ground_truth_masks = np.concatenate(ground_truth_masks, axis=0)
bounding_boxes = np.concatenate(bboxes, axis=0)

In [ ]:
results_zero_shot = []

for index in range(len(test_dataset)):
        mask = ground_truth_masks[index]
        sam_seg = predicted_masks[index]

        sam_seg_prob = torch.sigmoid(torch.tensor(sam_seg))
        # convert soft mask to hard mask
        sam_seg_prob = sam_seg_prob.cpu().numpy().squeeze()
        sam_seg = (sam_seg_prob > 0.5).astype(np.uint8)

        # Calculate IoU
        iou, intersection, union = calculate_iou(mask, sam_seg)

        results_zero_shot.append({'mask': mask,
                                  'calculated_mask': sam_seg, 
                                  'intersection': intersection.cpu().squeeze().numpy(), 
                                  'union': union.cpu().squeeze().numpy(), 
                                  'iou': iou.cpu().numpy(),
                                  'empty': False, 
                                  'bbox': bounding_boxes[index],
                                  'image_id': index,
                                  })

In [ ]:
#remove exemples with bounding boxes (0,0,512,512)
results_zero_shot_copy = results_zero_shot.copy()
results_zero_shot = [res for res in results_zero_shot if not (res['bbox'] == [0, 0, 1024, 1024]).all()]
results_zero_shot_general = [res for res in results_zero_shot_copy if (res['bbox'] == [0, 0, 1024, 1024]).all()]

results, metrics =calculate_pixel_based_metrics(results_zero_shot)

print(metrics)

results, metrics =calculate_pixel_based_metrics(results_zero_shot_general)

print(metrics)

results_zero_shot = results_zero_shot_copy

In [ ]:
# Create a DataFrame from the list of dictionaries
df_finetune = pd.DataFrame(results_zero_shot)

# Save the DataFrame to a file (optional)
#df_finetune.to_pickle('dataframe_name.pkl')

In [ ]:
# Filter out rows with None IoU values
df_filtered = df_finetune.dropna(subset=['iou'])


# Create a boolean mask for rows with bbox (0,0,512,512) using apply
mask_bbox = df_filtered['bbox'].apply(
    lambda b: np.all(np.array(b) == np.array((0, 0, 1024, 1024)))
)

In [ ]:
# Filter the DataFrame to exclude rows with bbox (0,0,512,512)

df_filtered = df_filtered[~mask_bbox]

In [ ]:
#
df_filtered = df_filtered[mask_bbox]

In [ ]:


# Extract IoU values from the filtered DataFrame
iou_values = df_filtered['iou'].tolist()
#iou_values = individual_ious

# Calculate the average IoU
average_iou = sum(iou_values) / len(iou_values)

# Create a bar chart
plt.figure(figsize=(10, 6))
plt.bar(range(len(iou_values)), iou_values, color='blue')
plt.xlabel('Sample Index')
plt.ylabel('IoU')
plt.title('IoU for Baseline (Pretrain)')
plt.text(0.5, 0.95, f'Average IoU: {average_iou:.4f}', ha='center', va='center', transform=plt.gca().transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
plt.show()

# Create a line plot
plt.figure(figsize=(10, 6))
plt.plot(range(len(iou_values)), iou_values, marker='o', linestyle='-', color='blue')
plt.xlabel('Sample Index')
plt.ylabel('IoU')
plt.title('IoU for Baseline (Pretrain)')
plt.text(0.5, 0.95, f'Average IoU: {average_iou:.4f}', ha='center', va='center', transform=plt.gca().transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
plt.show()

In [ ]:
# Assuming iou_values is a list of IoU scores
plt.figure(figsize=(10, 6))
plt.hist(iou_values, bins=20, color='blue', alpha=0.7)
plt.xlabel('IoU')
plt.ylabel('Frequency')
plt.title(f'IoU for Baseline (Pretrain)')
plt.text(0.5, 0.95, f'Average IoU: {average_iou:.4f}', ha='center', va='center', transform=plt.gca().transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Calculate the area of each mask
df_filtered['mask_area'] = df_filtered['mask'].apply(lambda mask: np.sum(mask))

# Filter out rows where the mask area is above 5000
df_filtered2 = df_filtered[df_filtered['mask_area'] <= 1000]

# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df_filtered2['mask_area'], df_filtered2['iou'], marker='o', color='blue')
plt.xlabel('Mask Area')
plt.ylabel('IoU')
plt.title('IoU vs. Mask Area')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Create a scatter plot with a logarithmic scale for the mask area axis
plt.figure(figsize=(10, 6))
plt.scatter(df_filtered['mask_area'], df_filtered['iou'], marker='o', color='blue')
plt.xscale('log')
plt.xlabel('Mask Area (log scale)')
plt.ylabel('IoU')
plt.title('IoU vs. Mask Area')
plt.grid(True)
plt.show()

In [ ]:
df_filtered['bbox'] = df_filtered['bbox'].apply(lambda b: np.squeeze(b) if np.array(b).ndim == 2 else b)

In [ ]:
import matplotlib.pyplot as plt

# Calculate the area of each bounding box (assumes bbox format is (x, y, w, h))
#df_filtered['bbox_area'] = df_filtered['bbox'].apply(lambda b: b[2] * b[3] if (b is not None and len(b) == 4) else None)

df_filtered['bbox_area'] = df_filtered['bbox'].apply(
    lambda b: (b[2] - b[0]) * (b[3] - b[1]) / 4 if (b is not None and len(b) == 4) else None
)

# Compute the ratio of bounding box area to mask area
df_filtered['area_ratio'] = df_filtered['mask_area'] / df_filtered['bbox_area'] 

# Drop rows with missing values in area_ratio or IoU to ensure clean plotting
df_plot = df_filtered.dropna(subset=['area_ratio', 'iou'])

plt.figure(figsize=(10, 6))
plt.scatter(df_plot['area_ratio'], df_plot['iou'], marker='o', color='blue')
plt.xlabel('Bounding Box Area / Mask Area')
plt.ylabel('IoU')
plt.title('IoU vs. Area Ratio')
plt.grid(True)
plt.show()

# Assume area_ratio and iou have been computed as before and stored in df_plot
plt.figure(figsize=(10, 6))
plt.scatter(df_plot['area_ratio'], df_plot['iou'], marker='o', color='blue')
plt.xscale('log')
plt.xlabel('Bounding Box Area / Mask Area (log scale)')
plt.ylabel('IoU')
plt.title('IoU vs. Area Ratio')
plt.grid(True)
plt.show()

In [ ]:
import cv2

# Function to count the number of disconnected regions (avalanches) in a mask.
# Assumes mask is a binary or grayscale image.
def count_avalanches(mask):
    # Ensure the mask is binary: any non-zero pixel is considered part of an avalanche.
    binary_mask = (mask > 0).astype(np.uint8)
    # cv2.connectedComponents returns (num_labels, labels); subtract 1 for the background.
    num_labels, _ = cv2.connectedComponents(binary_mask)
    return num_labels - 1  # subtract one for the background

# Compute the number of avalanches for each mask and store in a new column.
# This assumes the 'mask' column in df_filtered contains the mask array.
df_filtered['num_avalanches'] = df_filtered['mask'].apply(lambda m: count_avalanches(m) if m is not None else None)

# Drop rows with missing values in num_avalanches or IoU.
df_plot = df_filtered.dropna(subset=['num_avalanches', 'iou'])

plt.figure(figsize=(10, 6))
plt.scatter(df_plot['num_avalanches'], df_plot['iou'], marker='o', color='blue')
plt.xlabel('Number of Avalanches in Mask')
plt.ylabel('IoU')
plt.title('IoU vs. Number of Avalanches')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

mask_area_threshold = 100  # set your desired threshold

# Filter out rows where mask_area or IoU are missing and then select only those with mask_area above the threshold.
df_filtered_threshold = df_filtered.dropna(subset=['mask_area', 'iou'])
df_filtered_threshold = df_filtered_threshold[df_filtered_threshold['mask_area'] > mask_area_threshold]

# Extract IoU values from the filtered DataFrame
iou_values = df_filtered_threshold['iou'].tolist()

# Calculate the average IoU (ensure there is at least one value)
average_iou = sum(iou_values) / len(iou_values) if iou_values else 0

# Create a bar chart
plt.figure(figsize=(10, 6))
plt.bar(range(len(iou_values)), iou_values, color='blue')
plt.xlabel('Sample Index')
plt.ylabel('IoU')
plt.title(f'IoU Values for Samples with Mask Area > {mask_area_threshold}')
plt.text(0.5, 0.95, f'Average IoU: {average_iou:.4f}', ha='center', va='center', transform=plt.gca().transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
plt.show()

# Create a line plot
plt.figure(figsize=(10, 6))
plt.plot(range(len(iou_values)), iou_values, marker='o', linestyle='-', color='blue')
plt.xlabel('Sample Index')
plt.ylabel('IoU')
plt.title(f'IoU Values for Samples with Mask Area > {mask_area_threshold}')
plt.text(0.5, 0.95, f'Average IoU: {average_iou:.4f}', ha='center', va='center', transform=plt.gca().transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
plt.show()

# Assuming iou_values is a list of IoU scores
plt.figure(figsize=(10, 6))
plt.hist(iou_values, bins=20, color='blue', alpha=0.7)
plt.xlabel('IoU')
plt.ylabel('Frequency')
plt.title(f'Histogram of IoU Scores (Mask Area > {mask_area_threshold})')
plt.text(0.5, 0.95, f'Average IoU: {average_iou:.4f}', ha='center', va='center', transform=plt.gca().transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
plt.show()

In [ ]:
print(df_filtered['mask_area'].min())

In [ ]:
df_filtered['mask_area'] = df_filtered['mask'].apply(lambda mask: np.sum(np.array(mask, dtype=np.float32)))

In [ ]:
# Convert columns to numpy arrays with a numeric dtype
mask_area_numeric = np.asarray(df_filtered['mask_area'].values, dtype=np.float32)
iou_numeric = np.asarray(df_filtered['iou'].values, dtype=np.float32)

corr, p_value = calculate_correlation(mask_area_numeric, iou_numeric)
print(f"Correlation between mask area and IoU: {corr}, p-value: {p_value}")

In [ ]:
# Ensure no zeros (to avoid log issues) by adding a small constant if needed.
epsilon = 1e-8
mask_area_log = np.log(mask_area_numeric + epsilon)

corr, p_value = calculate_correlation(mask_area_log, iou_numeric)
print(f"Correlation on log-transformed values: {corr}, p-value: {p_value}")

In [ ]:
# Convert columns to numpy arrays with a numeric dtype
num_avalanches_numric = np.asarray(df_filtered['num_avalanches'].values, dtype=np.float32)
iou_numeric = np.asarray(df_filtered['iou'].values, dtype=np.float32)

corr, p_value = calculate_correlation(num_avalanches_numric, iou_numeric)
print(f"Correlation between number of avalanches and IoU: {corr}, p-value: {p_value}")

In [ ]:
# Convert columns to numpy arrays with a numeric dtype
area_ratio_numeric = np.asarray(df_filtered['area_ratio'].values, dtype=np.float32)
iou_numeric = np.asarray(df_filtered['iou'].values, dtype=np.float32)

corr, p_value = calculate_correlation(area_ratio_numeric, iou_numeric)
print(f"Correlation between area ratio and IoU: {corr}, p-value: {p_value}")

In [ ]:
# Ensure no zeros (to avoid log issues) by adding a small constant if needed.
epsilon = 1e-8
area_ratio_log = np.log(area_ratio_numeric + epsilon)

corr, p_value = calculate_correlation(area_ratio_log, iou_numeric)
print(f"Correlation on log-transformed values: {corr}, p-value: {p_value}")

In [ ]:
# Function to find bounding boxes for each group of disconnected white pixels
def find_bounding_boxes(mask):
    # Ensure the mask is an 8-bit image.
    if mask.dtype != "uint8":
        # If mask values are in range 0-1, scale them by 255
        if mask.max() <= 1:
            mask_uint8 = (mask * 255).astype('uint8')
        else:
            mask_uint8 = mask.astype('uint8')
    else:
        mask_uint8 = mask

    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Compute bounding boxes for each contour and convert (x, y, w, h) to (x_min, y_min, x_max, y_max)
    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append([x, y, x + w, y + h])
    return bounding_boxes

In [ ]:
treshold = 0.3

for res in results_zero_shot:
    false_positives = []
    false_negatives = []
    # The predicted and ground truth avalanche masks
    pred_mask = res['calculated_mask']
    true_mask = res['mask']

    original_bbox = find_bounding_boxes(true_mask)
    pred_bbox = find_bounding_boxes(pred_mask)

    for bbox in original_bbox:
        predicted = False
        for pred in pred_bbox:

            # Assuming each bbox is [x_min, y_min, x_max, y_max]
            x_left = max(bbox[0], pred[0])
            y_top = max(bbox[1], pred[1])
            x_right = min(bbox[2], pred[2])
            y_bottom = min(bbox[3], pred[3])

            # Calculate the overlap area
            if x_right < x_left or y_bottom < y_top:
                overlap_area = 0
            else:
                overlap_area = (x_right - x_left) * (y_bottom - y_top)

            overlap = overlap_area / ((bbox[2] - bbox[0]) * (bbox[3] - bbox[1]) )#+ (pred[2] - pred[0]) * (pred[3] - pred[1]) - overlap_area)
            if overlap > treshold:
                predicted = True
                break
        if not predicted:
            false_negatives.append(bbox)
    for bbox in pred_bbox:
        original = False
        for originalbbox in original_bbox:
            # Assuming each bbox is [x_min, y_min, x_max, y_max]
            x_left = max(bbox[0], originalbbox[0])
            y_top = max(bbox[1], originalbbox[1])
            x_right = min(bbox[2], originalbbox[2])
            y_bottom = min(bbox[3], originalbbox[3])

            # Calculate the overlap area
            if x_right < x_left or y_bottom < y_top:
                overlap_area = 0
            else:
                overlap_area = (x_right - x_left) * (y_bottom - y_top)

            overlap = overlap_area / ((bbox[2] - bbox[0]) * (bbox[3] - bbox[1]) )#+ (originalbbox[2] - originalbbox[0]) * (originalbbox[3] - originalbbox[1]) - overlap_area)
            if overlap > treshold:
                original = True
                break
        if not original:
            false_positives.append(bbox)
            
    # Add the results into the current dictionary element
    res['false_negatives'] = false_negatives
    res['false_positives'] = false_positives
    res['percentage_false_negatives'] = len(false_negatives) / len(original_bbox) if len(original_bbox) > 0 else 0
    res['percentage_false_positives'] = len(false_positives) / len(pred_bbox) if len(pred_bbox) > 0 else 0
        


In [ ]:
res = results_zero_shot[0]

pred_mask = res['calculated_mask']
true_mask = res['mask']

print("Shape of predicted mask:", pred_mask.shape)
print("Shape of true mask:", true_mask.shape)

In [ ]:
treshold = 0.3

for res in results_zero_shot:
    false_positives = []
    false_negatives = []
    # The predicted and ground truth avalanche masks
    pred_mask = res['calculated_mask']
    true_mask = res['mask']

    original_bbox = find_bounding_boxes(true_mask)
    pred_bbox = find_bounding_boxes(pred_mask)

    for bbox in original_bbox:
        predicted = False
        for pred in pred_bbox:

            pred_mask_local = np.zeros_like(pred_mask)
            pred_mask_local[pred[1]:pred[3], pred[0]:pred[2]] = pred_mask[pred[1]:pred[3], pred[0]:pred[2]]

            original_mask_local = np.zeros_like(true_mask)
            original_mask_local[bbox[1]:bbox[3], bbox[0]:bbox[2]] = true_mask[bbox[1]:bbox[3], bbox[0]:bbox[2]]

            # Calculate the overlap area
            overlap_area = np.sum(np.logical_and(pred_mask_local, original_mask_local))
            original_area = np.sum(original_mask_local)

            overlap = overlap_area / original_area if original_area > 0 else 0
            if overlap > treshold:
                predicted = True
                break
        if not predicted:
            false_negatives.append(bbox)
    for bbox in pred_bbox:
        original = False
        for originalbbox in original_bbox:
            
            pred_mask_local = np.zeros_like(pred_mask)
            pred_mask_local[bbox[1]:bbox[3], bbox[0]:bbox[2]] = pred_mask[bbox[1]:bbox[3], bbox[0]:bbox[2]]

            original_mask_local = np.zeros_like(true_mask)
            original_mask_local[originalbbox[1]:originalbbox[3], originalbbox[0]:originalbbox[2]] = true_mask[originalbbox[1]:originalbbox[3], originalbbox[0]:originalbbox[2]]

            # Calculate the overlap area
            overlap_area = np.sum(np.logical_and(pred_mask_local, original_mask_local))
            pred_area = np.sum(pred_mask_local)

            overlap = overlap_area / pred_area if pred_area > 0 else 0
            if overlap > treshold:
                original = True
                break
        if not original:
            false_positives.append(bbox)
            
    # Add the results into the current dictionary element
    res['false_negatives'] = false_negatives
    res['false_positives'] = false_positives
    res['percentage_false_negatives'] = len(false_negatives) / len(original_bbox) if len(original_bbox) > 0 else 0
    res['percentage_false_positives'] = len(false_positives) / len(pred_bbox) if len(pred_bbox) > 0 else 0
        


In [ ]:
#remove exemples with bounding boxes (0,0,512,512)
results_zero_shot_copy = results_zero_shot.copy()
results_zero_shot = [res for res in results_zero_shot if not (res['bbox'] == [0, 0, 1024, 1024]).all()]
results_zero_shot_general = [res for res in results_zero_shot_copy if (res['bbox'] == [0, 0, 1024, 1024]).all()]

In [ ]:
results_zero_shot = results_zero_shot_copy

In [ ]:
# New cell: Plot the percentage of false positives and false negatives with averages
import matplotlib.pyplot as plt
import numpy as np



# Extract percentages from the results
false_negatives_pct = [res.get('percentage_false_negatives', 0) for res in results_zero_shot]
false_positives_pct = [res.get('percentage_false_positives', 0) for res in results_zero_shot]
indices = np.arange(len(results_zero_shot))

# Compute average percentages
avg_false_negatives = np.mean(false_negatives_pct)
avg_false_positives = np.mean(false_positives_pct)

plt.figure(figsize=(12, 6))
plt.bar(indices - 0.15, false_negatives_pct, width=0.3, color='red', label='False Negatives')
plt.bar(indices + 0.15, false_positives_pct, width=0.3, color='blue', label='False Positives')

# Plot average lines
plt.axhline(avg_false_negatives, color='darkred', linestyle='--', 
            label=f'Avg False Negatives: {avg_false_negatives:.2f}')
plt.axhline(avg_false_positives, color='darkblue', linestyle='--', 
            label=f'Avg False Positives: {avg_false_positives:.2f}')

plt.xlabel('Sample Index')
plt.ylabel('Percentage')
plt.title(f'Percentage of False Negatives and False Positives per Sample (Baseline, Treshold = {treshold})')
plt.legend()
plt.show()

In [ ]:
# New cell: Plot the percentage of false positives and false negatives with averages
import matplotlib.pyplot as plt
import numpy as np

# Extract percentages from the results
false_negatives_pct = [res.get('percentage_false_negatives', 0) for res in results_zero_shot_general]
false_positives_pct = [res.get('percentage_false_positives', 0) for res in results_zero_shot_general]
indices = np.arange(len(results_zero_shot_general))

# Compute average percentages
avg_false_negatives = np.mean(false_negatives_pct)
avg_false_positives = np.mean(false_positives_pct)

plt.figure(figsize=(12, 6))
plt.bar(indices - 0.15, false_negatives_pct, width=0.3, color='red', label='False Negatives')
plt.bar(indices + 0.15, false_positives_pct, width=0.3, color='blue', label='False Positives')

# Plot average lines
plt.axhline(avg_false_negatives, color='darkred', linestyle='--', 
            label=f'Avg False Negatives: {avg_false_negatives:.2f}')
plt.axhline(avg_false_positives, color='darkblue', linestyle='--', 
            label=f'Avg False Positives: {avg_false_positives:.2f}')

plt.xlabel('Sample Index')
plt.ylabel('Percentage')
plt.title('Percentage of False Negatives and False Positives per Sample')
plt.legend()
plt.show()

In [ ]:
# New cell: Visualization of samples with high error percentages including the associated image 
# and original bounding box (green) in ground truth and prediction

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import numpy as np

# Define a threshold for high error (e.g., 50% error for this example)
threshold = 0.4

# Filter samples with either false negative or false positive percentage above the threshold
high_error_results = [
    res for res in results_zero_shot 
    if res.get('percentage_false_negatives', 0) > threshold or res.get('percentage_false_positives', 0) > threshold
]

print(f"Number of high error samples: {len(high_error_results)}")

In [ ]:
test_dataset_test = load_from_disk('/home/gelato/Avalanche-Segmentation-with-Sam/code/dataprocessing/datasetTestDEMFloat')

In [ ]:
for idx, res in enumerate(high_error_results):

    if idx >= 10:  # Limit visualization to the first 10 high error samples
        break
    
    # Create a row of 3 subplots:
    # Left: Ground Truth with false negatives and original bbox
    # Middle: Associated original image
    # Right: Predicted mask with false positives and original bbox
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
    # --- Left: Ground truth mask with false negative outlines and original bounding box in green --- 
    mask = res['mask']
    if mask.ndim == 2:
        gt_img = (mask * 255).astype(np.uint8)
        gt_img = cv2.cvtColor(gt_img, cv2.COLOR_GRAY2RGB)
    else:
        gt_img = mask.copy()
    ax1.imshow(gt_img)
    ax1.set_title("Ground Truth with False Negatives")
    
    # Draw the original bounding box in green
    
    orig_bbox = res['bbox']/2
    rect_orig = patches.Rectangle((orig_bbox[0], orig_bbox[1]),
                                    orig_bbox[2] - orig_bbox[0],                                   
                                    orig_bbox[3] - orig_bbox[1],
                                    linewidth=2,
                                    edgecolor='g',
                                    facecolor='none')
    rect_orig2 = patches.Rectangle((orig_bbox[0], orig_bbox[1]),
                                    orig_bbox[2] - orig_bbox[0],                                   
                                    orig_bbox[3] - orig_bbox[1],
                                    linewidth=2,
                                    edgecolor='g',
                                    facecolor='none')
    
    ax1.add_patch(rect_orig)
    
    # Draw false negative bounding boxes in red
    for bbox in res.get('false_negatives', []):
        rect_fn = patches.Rectangle((bbox[0], bbox[1]),
                                    bbox[2] - bbox[0],
                                    bbox[3] - bbox[1],
                                    linewidth=2,
                                    edgecolor='r',
                                    facecolor='none')
        ax1.add_patch(rect_fn)
   
    # --- Middle: Associated original image ---
    orig_image =  test_dataset_test[res['image_id']]['image'] if 'image_id' in res else None
    if not isinstance(orig_image, np.ndarray):
        orig_image = np.array(orig_image)
    orig_image_disp = (orig_image * 255).astype(np.uint8)
    ax2.imshow(orig_image_disp)
    ax2.set_title("Original Image")
    
    # --- Right: Predicted mask with false positive outlines and original bounding box in green ---
    pred_mask = res['calculated_mask']
    if pred_mask.ndim == 2:
        pred_img = (pred_mask * 255).astype(np.uint8)
        pred_img = cv2.cvtColor(pred_img, cv2.COLOR_GRAY2RGB)
    else:
        pred_img = pred_mask.copy()
    ax3.imshow(pred_img)
    ax3.set_title("Prediction with False Positives")
    
    # Draw the original bounding box in green 
    ax3.add_patch(rect_orig2)
    
    # Draw false positive bounding boxes in blue
    for bbox in res.get('false_positives', []):
        rect_fp = patches.Rectangle((bbox[0], bbox[1]),
                                    bbox[2] - bbox[0],
                                    bbox[3] - bbox[1],
                                    linewidth=2,
                                    edgecolor='b',
                                    facecolor='none')
        ax3.add_patch(rect_fp)
    
    # Hide axis from all subplots for clarity
    for ax in (ax1, ax2, ax3):
        ax.axis('off')
        
    plt.tight_layout()
    save_path = 'images/adapters/falsepn/normalbox'
    os.makedirs(save_path, exist_ok=True)
    #plt.savefig(f'{save_path}/{idx}.png')
    plt.show()
    plt.close(fig)  # Close the figure to free memory

In [ ]:
# New cell: Visualization of samples with high error percentages including the associated image 
# and original bounding box (green) in ground truth and prediction

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import numpy as np

# Define a threshold for high error (e.g., 50% error for this example)
threshold = 0.5

# Filter samples with either false negative or false positive percentage above the threshold
high_error_results = [
    res for res in results_zero_shot_general
    #if res.get('percentage_false_negatives', 0) > threshold or res.get('percentage_false_positives', 0) > threshold
]

print(f"Number of high error samples: {len(high_error_results)}")

In [ ]:
plt.rcParams['font.size'] = 20
for idx, res in enumerate(high_error_results):

    if idx >= 30:  # Limit visualization to the first 30 high error samples
        break
    if idx <15:
        continue
    
    # Create a row of 3 subplots:
    # Left: Ground Truth with false negatives and original bbox
    # Middle: Associated original image
    # Right: Predicted mask with false positives and original bbox
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
    # --- Left: Ground truth mask with false negative outlines and original bounding box in green --- 
    mask = res['mask']
    if mask.ndim == 2:
        gt_img = (mask * 255).astype(np.uint8)
        gt_img = cv2.cvtColor(gt_img, cv2.COLOR_GRAY2RGB)
    else:
        gt_img = mask.copy()
    ax1.imshow(gt_img)
    ax1.set_title("Ground Truth")
    
    # Draw the original bounding box in green
    
    orig_bbox = res['bbox'][0]/2
    orig_bbox = orig_bbox - 0.5
    rect_orig = patches.Rectangle((orig_bbox[0], orig_bbox[1]),
                                    orig_bbox[2] - orig_bbox[0],                                   
                                    orig_bbox[3] - orig_bbox[1],
                                    linewidth=3,
                                    edgecolor='g',
                                    facecolor='none')
    rect_orig2 = patches.Rectangle((orig_bbox[0], orig_bbox[1]),
                                    orig_bbox[2] - orig_bbox[0],                                   
                                    orig_bbox[3] - orig_bbox[1],
                                    linewidth=3,
                                    edgecolor='g',
                                    facecolor='none')
    
    ax1.add_patch(rect_orig)
   
    # --- Middle: Associated original image ---
    orig_image =  test_dataset_test[res['image_id']]['image'] if 'image_id' in res else None
    if not isinstance(orig_image, np.ndarray):
        orig_image = np.array(orig_image)
    orig_image_disp = (orig_image * 255).astype(np.uint8)
    #Substitute DEM with first channel of the image
    orig_image_disp[:,:,2] = orig_image_disp[:, :, 0]
    
    ax2.imshow(orig_image_disp)
    ax2.set_title("Image")
    
    # --- Right: Predicted mask with false positive outlines and original bounding box in green ---
    pred_mask = res['calculated_mask']
    if pred_mask.ndim == 2:
        pred_img = (pred_mask * 255).astype(np.uint8)
        pred_img = cv2.cvtColor(pred_img, cv2.COLOR_GRAY2RGB)
    else:
        pred_img = pred_mask.copy()
    ax3.imshow(pred_img)
    ax3.set_title("Prediction")
    
    # Draw the original bounding box in green 
    ax3.add_patch(rect_orig2)
    
    # Draw false positive bounding boxes in blue
    for bbox in res.get('false_positives', []):
        rect_fp = patches.Rectangle((bbox[0], bbox[1]),
                                    bbox[2] - bbox[0],
                                    bbox[3] - bbox[1],
                                    linewidth=2,
                                    edgecolor='b',
                                    facecolor='none')
        ax3.add_patch(rect_fp)
    
    # Hide axis from all subplots for clarity
    for ax in (ax1, ax2, ax3):
        ax.axis('off')
        
    plt.tight_layout()
    save_path = 'images/adapters/falsepn/fullimagebox'
    os.makedirs(save_path, exist_ok=True)
    #plt.savefig(f'{save_path}/{idx}.png')
    plt.show()
    plt.close(fig)  # Close the figure to free memory